In [45]:
def computePatternRecallFromTuple(sg, total_eids, eids2Sup, eid2recall, eidsFeatureCount):
      tmp_sum = 0.0
      for eid in range(total_eids):
        if eids2Sup[eid] > 0:
          tmp_sum += eid2recall.get(eid, 0) * eidsFeatureCount[eid][sg] / eids2Sup[eid]
      return tmp_sum

def computeTupleRecallFromPattern(eid, extr_patterns, eidsFeatureCount, patterns2Sup, pattern2recall):
      tmp_sum = 0.0
      for sg in extr_patterns:
        tmp_sum += pattern2recall.get(sg, 0) * eidsFeatureCount[eid][sg] / patterns2Sup[sg]
      return tmp_sum

def doInferenceRecallExtPatternOnly(extr_patterns, pattern2recall, total_eids, eid2recall, 
                                    eids2Sup, eidsFeatureCount, patterns2Sup):
      for sg in extr_patterns:
        pattern2recall[sg] = computePatternRecallFromTuple(sg, total_eids, eids2Sup, 
                                                           eid2recall, eidsFeatureCount)
      for eid in range(total_eids):
        eid2recall[eid] = computeTupleRecallFromPattern(eid, extr_patterns, eidsFeatureCount, 
                                                        patterns2Sup, pattern2recall)

In [46]:
def computePatternPrecisionFromTuple(p, pattern2eids, eidsFeatureCount, 
                                     pattern2precision, eid2precision, patterns2Sup):
      tmp_sum = 0.0
      for eid in pattern2eids[p]:
        sup = eidsFeatureCount[eid][p]
        tmp_sum += eid2precision[eid]*sup
      total = patterns2Sup[p]
      if total > 0:
        return tmp_sum / total
      else: return 0.0

def computeTuplePrecisionFromPattern(eid, groundTruthEids, eid2precision, 
                                     extr_patterns, eidsFeatureCount, eid2patterns, 
                                     pattern2precision, eids2Sup):
      if eid in groundTruthEids:
        return eid2precision[eid]
      tmp_sum = 0.0
      for sg in eid2patterns[eid]:
        sup = eidsFeatureCount[eid][sg]
        tmp_sum += pattern2precision[sg]*sup
      total = eids2Sup[eid]
      if total > 0:
        return tmp_sum / total
      else: return 0.0

def doInferencePrecExtPatternOnly(extr_patterns, pattern2eids, eidsFeatureCount, 
                                  pattern2precision, eid2precision, patterns2Sup, 
                                  total_eids, groundTruthEids, eid2patterns, eids2Sup):
      for sg in extr_patterns:
        pattern2precision[sg] = computePatternPrecisionFromTuple(sg, pattern2eids, eidsFeatureCount, 
                                                                 pattern2precision, eid2precision, patterns2Sup)

      for eid in range(total_eids):
        if eid not in groundTruthEids:
          eid2precision[eid] = computeTuplePrecisionFromPattern(eid, groundTruthEids, eid2precision,extr_patterns, 
                                                                eidsFeatureCount, eid2patterns, pattern2precision, 
                                                                eids2Sup)

In [47]:
# expand the set of seedEntities and return eids by order, excluding seedEntities (original children)
def prDualRank(seedEidsWithConfidence, negativeSeedEids, eid2patterns, pattern2eids, eidAndPattern2strength,
             eid2types, type2eids, eidAndType2strength, eid2ename, eidsFeatureCount, eids2Sup, patterns2Sup, 
             FLAGS_VERBOSE=False, FLAGS_DEBUG=False):
      ''' 

      :param seedEidsWithConfidence: a list of [eid (int), confidence_score (float)]
      :param negativeSeedEids: a set of eids (int) that should not be included
      :param eid2patterns:
      :param pattern2eids:
      :param eidAndPattern2strength:
      :param eid2types:
      :param type2eids:
      :param eidAndType2strength:
      :param eid2ename:

      :return: P and T sorted by precision and recall
      '''
      # print(patterns2Sup)
      seedEids = [key for key, val in seedEidsWithConfidence.items()]
      groundTruthEids = [key for key, val in seedEidsWithConfidence.items()]
      eid2confidence = {key:val for key, val in seedEidsWithConfidence.items()}

      ## Cache the seedEids for later use
      cached_seedEids = set([ele for ele in seedEids])
      if FLAGS_VERBOSE:
        print('Seed set:')
        for eid in seedEids:
          print(eid, eid2ename[eid])
        print("[INFO] Start SetExpan")

    ## establish ground truth
    ##double but not int
      total_eids = len(eid2ename)
      eid2precision = {x: 0.0 for x in range(total_eids)}
      for eid in seedEids:
        eid2precision[eid] = 1.0
      tmp_sum = len(seedEids)
      eid2recall = {eid: eid2precision[eid]/tmp_sum for eid in seedEids}

      extr_patterns = list(pattern2eids.keys())

      pattern2precision = {sg: 0.0 for sg in extr_patterns}
      pattern2recall = {sg: 0.0 for sg in extr_patterns}


      iters = 0
      while iters < 100:
        iters += 1
        prev_seeds = set(seedEids)
        #start = time.time()

        #QuestP till convergence
        doInferencePrecExtPatternOnly(extr_patterns, pattern2eids, eidsFeatureCount, 
                                      pattern2precision, eid2precision, patterns2Sup, 
                                      total_eids, groundTruthEids, eid2patterns, eids2Sup)
        #QuestR till convergence
        doInferenceRecallExtPatternOnly(extr_patterns, pattern2recall, total_eids, 
                                        eid2recall, eids2Sup, eidsFeatureCount, patterns2Sup)


        #all_end = time.time()

        if FLAGS_DEBUG:
          print("End of iteration %s" % iters)

      expanded_eid_pre = sorted(eid2precision, key=eid2precision.__getitem__, reverse=True)
      expanded_eid_rec = sorted(eid2recall, key=eid2recall.__getitem__, reverse=True)
      expanded_pattern_pre = sorted(pattern2precision, key=pattern2precision.__getitem__, reverse=True)
      expanded_pattern_rec = sorted(pattern2recall, key=pattern2recall.__getitem__, reverse=True)

      return expanded_pattern_pre, expanded_pattern_rec, expanded_eid_pre, expanded_eid_rec, pattern2precision, pattern2recall, eid2precision, eid2recall

In [48]:
import spacy
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

In [49]:
from whoosh.qparser import QueryParser
import whoosh.index as index
from whoosh.index import open_dir
from whoosh.query import Every
from collections import defaultdict

In [50]:
# library for list manipulation
import itertools

# library for context graph
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [51]:
nlp = spacy.load('en_core_web_sm')

In [52]:
def extractor(document, *pattern):
    '''
    @param str file: text file for tuple extraction
    @param list pattern: extraction pattern
    A phrase extractor based on spacy
    '''
    phrases = []
    matcher = Matcher(nlp.vocab)
    matcher.add("pattern extraction", None, *pattern)
    doc = nlp(document)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        phrases.append(span.text)
    return phrases

In [53]:
def partition(file, size = 1000000):
    '''
    partition the input file into block with maximum size of 1000000, since SpaCy v2.x parser may have issues allocating memory with size larger than 1000000
    '''
    while True:
        data = file.read(size).lower()
        if not data:
            break
        yield data

In [54]:
def getPhrases(file, context_pattern):
    new_phrases = set()
    with open(file, 'r') as f:
        t = f.read().lower()
        matcher = Matcher(nlp.vocab)
        doc = nlp(t)
        for cp in context_pattern:
            matcher.add("extraction", None, cp)
            matches = matcher(doc)
            for match_id, start, end in matches:
                span = doc[start+2:end].text
                if span not in new_phrases:
                    new_phrases.add(span)
#                     print(span)
            matcher.remove("extraction")
    return new_phrases

In [89]:
def patternSearch(T_0, file):
    phrase_patterns = set()
    seed_pattern = [nlp(x) for x in T_0]
    phrase_matcher = PhraseMatcher(nlp.vocab)
    phrase_matcher.add('pattern search', None, *seed_pattern)
    # find occurrences of seed phrases
    with open(file, "r") as f:
        document = nlp(f.read().lower())
        matches = phrase_matcher(document)
        for match_id, start, end in matches:
            p = tuple((start, end))
            if p not in phrase_patterns:
                phrase_patterns.add(p)
    # find patterns around seed phrases
    unranked_patterns = []
    with open(file, "r") as f:
        text = nlp(f.read().lower())
        for phrase_pattern in phrase_patterns:
            start = phrase_pattern[0]
            end = phrase_pattern[1]
            if (text[start - 1].text == '\n'):
                continue
            # add context pattern 
            tmp = []
            for i in range(2, 0, -1):
                tmp.append({"TEXT": text[start - i].text})
            # add content pattern 
            span = text[start:end]
            for token in span:
                tmp.append({"POS": token.pos_})
            if tmp not in unranked_patterns:
                unranked_patterns.append(tmp)
                # print(tmp)
    unranked_phrases = list(getPhrases(file, unranked_patterns))
    
    phrase2id = {}
    for i in range(len(unranked_phrases)):
        phrase2id[unranked_phrases[i]] = i
#     print(phrase2id)
    
    id2phrase = {}
    for i in range(len(unranked_phrases)):
        id2phrase[i] = unranked_phrases[i]
#     print(id2phrase)
    
    id2pattern = {}
    for i in range(len(unranked_patterns)):
        id2pattern[i] = unranked_patterns[i]
#     print(id2pattern)
    
    seedIdwConfidence = {}
    for key, val in phrase2id.items():
        if key in T_0:
            seedIdwConfidence[val] = 0.0
#     print(seedIdwConfidence)
    
    id2patterns = defaultdict(set)
    pattern2ids = defaultdict(set)
    
    # build context graph
    context_graph = nx.Graph()
    # add tuples and patterns into graph
    for i in range(len(unranked_phrases)):
        node = 't' + str(i)
        context_graph.add_node(node, pos=(0, i))
    for i in range(len(unranked_patterns)):
        node = 'p' + str(i)
        context_graph.add_node(node, pos=(2, i))
    context_matrix = np.zeros((len(unranked_phrases), len(unranked_patterns)))
    # find c (t, p)
    with open(file, 'r') as f:
        t = f.read().lower()
        matcher = Matcher(nlp.vocab)
        doc = nlp(t)
        for i in range(len(unranked_patterns)):
            matcher.add("extraction", None, unranked_patterns[i])
            matches = matcher(doc)
            for match_id, start, end in matches:
                span = doc[start+2:end].text
                j = unranked_phrases.index(span)
                context_matrix[j, i] += 1
                id2patterns[j].add(i)
                pattern2ids[i].add(j)
            matcher.remove("extraction")
    # add context nodes into graph
    
#     print(pattern2ids)
#     print(id2patterns)
    
    id2sup = {key:len(val) for key, val in id2patterns.items()}
    pattern2sup = {key:len(val) for key, val in pattern2ids.items()}
    
#     print(id2sup)
#     print(pattern2sup)
    
    l1, l2, l3, l4, m1, m2, _, _ = prDualRank(seedIdwConfidence, [], id2patterns, pattern2ids, {},
             {}, {}, {}, id2phrase, context_matrix.tolist(), id2sup, pattern2sup, 
             FLAGS_VERBOSE=False, FLAGS_DEBUG=False)
    
    expanded_pattern_pre = [unranked_patterns[i] for i in l1] 
    expanded_pattern_rec = [unranked_patterns[i] for i in l2] 
    expanded_eid_pre = [unranked_phrases[i] for i in l3]
    expanded_eid_rec = [unranked_phrases[i] for i in l4]
    
    print(expanded_pattern_pre)
    print(expanded_pattern_rec)
    print(expanded_eid_pre)
    print(expanded_eid_rec)
    
    print(m1)
    print(m2)
    
    c_count = 0
    for i in range(context_matrix.shape[0]):
        for j in range(context_matrix.shape[1]):
            if context_matrix[i, j] != 0:
                occur = context_matrix[i, j]
                node_t = 't' + str(i)
                node_p = 'p' + str(j)
                node_c = 'c' + str(c_count)
                c_count += 1
                context_graph.add_node(node_c, pos=(1, c_count))
                context_graph.add_edge(node_t, node_c, weight=occur)
                context_graph.add_edge(node_c, node_p, weight=occur)
    # draw context graph
#     pos=nx.get_node_attributes(context_graph,'pos')
#     nx.draw(context_graph, pos, with_labels=True)
#     labels = nx.get_edge_attributes(context_graph, 'weight')
#     nx.draw_networkx_edge_labels(context_graph,pos,edge_labels=labels)
    # return patterns
    return expanded_pattern_pre, expanded_pattern_rec, expanded_eid_pre, expanded_eid_rec, m1, m2, unranked_patterns

In [90]:
def tuple_search(P_e_recall, P_e_precision, pattern2precision, pattern2recall, unranked_patterns, file):
    pattern2fscore = {}
    for i in range(len(unranked_patterns)):
        recall = pattern2recall[i]
        precision = pattern2precision[i]
        fscore = ((2 * recall * precision) / (recall + precision))
        pattern2fscore[i] = fscore
    sorted_patterns_ids = sorted(pattern2fscore, key=pattern2fscore.__getitem__, reverse=True)
    sorted_patterns = [unranked_patterns[i] for i in sorted_patterns_ids]
    print(sorted_patterns[0:10])
    sorted_patterns = sorted_patterns[0:10]
    new_phrases = list(getPhrases(file, sorted_patterns))
    print(new_phrases)
    return sorted_patterns, new_phrases

In [57]:
seed = set(["machine learning", "query optimization", "RSA encryption", "tensor algebra"]) # set(['multimedia data types', 'database system', 'cryptographic algorithm'])
# f = open("large.txt", "r")
# chunks = partition(f, size = 1000000)
# f = open("new1.txt", "w")
# for line in list(chunks)[0]:
#     f.write(line)
list1, list2, list3, list4, m1, m2, l5 = patternSearch(seed, 'test.txt')
# print(context_p)
# print(new_phrases)

[[{'TEXT': 'applications'}, {'TEXT': 'of'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'inference'}, {'TEXT': 'and'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'based'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'transitions'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'curves'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': '\n'}, {'TEXT': 'online'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'analysis'}, {'TEXT': 'using'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'stream'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'quantum'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': '-'}, {'TEXT': 'a'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'encyclopedia'}, {'TEXT': 'of'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'encyclopedia'}, {'TEXT': 'of'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]]
[[{'TEXT': 'applications'}, {'TEXT': 'of'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'ba

In [81]:
l_patt = tuple_search(list1, list2, m1, m2, l5, "test.txt")

[[{'TEXT': 'applications'}, {'TEXT': 'of'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'inference'}, {'TEXT': 'and'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'based'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'transitions'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'curves'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': '\n'}, {'TEXT': 'online'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'analysis'}, {'TEXT': 'using'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}]]
['systems using', 'information theory', 'job scheduling', 'machine learning', 'face recognition']


In [84]:
new_seed = set(['systems using', 'information theory', 'job scheduling', 'machine learning', 'face recognition', "machine learning", "query optimization", "RSA encryption", "tensor algebra"])

In [86]:
l_1, l_2, l_3, l_4, m_1, m_2, l_5 = patternSearch(new_seed, 'test.txt')

[[{'TEXT': '-'}, {'TEXT': 'based'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'methods'}, {'TEXT': 'for'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'encyclopedia'}, {'TEXT': 'of'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}], [{'TEXT': 'three'}, {'TEXT': 'dimensional'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'inference'}, {'TEXT': 'and'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'infrared'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'transitions'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'for'}, {'TEXT': 'online'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'curves'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'analysis'}, {'TEXT': 'using'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'applications'}, {'TEXT': 'of'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'based'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': '\n'}, {'TEXT': 'online'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'aware'}, {'TEXT

In [88]:
lpatt = tuple_search(l_1, l_2, m_1, m_2, l_5, "test.txt")

[[{'TEXT': '-'}, {'TEXT': 'based'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'methods'}, {'TEXT': 'for'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'encyclopedia'}, {'TEXT': 'of'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}], [{'TEXT': 'three'}, {'TEXT': 'dimensional'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'inference'}, {'TEXT': 'and'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'infrared'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'transitions'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'for'}, {'TEXT': 'online'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'curves'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'VERB'}], [{'TEXT': 'analysis'}, {'TEXT': 'using'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}]]
['internet routers', 'storage devices', 'photo album', 'motion picture', 'dos defenses', 'resource retrieval', 'parallel algorithms', 'enterprise architectures', 'multicriteria decision', 'lossless image', 'recommendation systems', 'simulation mo

In [91]:
new_new_seed = set(['systems using', 'information theory', 'job scheduling', 'machine learning', 'face recognition', "machine learning", "query optimization", "RSA encryption", "tensor algebra", 'internet routers', 'storage devices', 'photo album', 'motion picture', 'dos defenses', 'resource retrieval', 'parallel algorithms', 'enterprise architectures', 'multicriteria decision', 'lossless image', 'recommendation systems', 'simulation models', 'television services', 'multimedia storage', 'intrusion detection', 'information retrieval', 'computational neuroscience', 'object retrieval', 'matrix quadtree', 'communication systems', 'computer vision', 'approximation algorithms', 'motion coordination', 'target tracking', 'hardware platforms', 'web server', 'message authentication', 'data mining', 'service differentiation', 'optimization techniques', 'testing tool', 'hand gesture', 'reinforcement\\break learning', 'information assurance', 'shape retrieval', 'routing algorithm', 'assessment system', 'face verification', 'video compression', 'security policy', 'model checking', 'botnet detection', 'sql solutions', 'copyright protection', 'software engineering', 'knowledge management', 'buffer overflows', 'security protocols', 'multimedia retrieval', 'image indexing', 'asset compiler', 'process analysis', 'belief revision', 'internet pornography', 'search engine', 'knowledge integration', 'feature extraction', 'access control', 'data structures', 'planning techniques', 'reinforcement learning', 'user interfaces', 'face identification', 'optimization problems', 'query processing', 'machine learning', 'resource allocation', 'face recognition', 'authorization languages'])

In [92]:
l_one, l_two, l_three, l_four, m_one, m_two, l_five = patternSearch(new_new_seed, 'test.txt')

[[{'TEXT': '-'}, {'TEXT': 'infinite'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'k'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'information'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'data'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'geographic'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'geographic'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'graph'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'temporal'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'aware'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'intelligent'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'spatial'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}], [{'TEXT': 'networks'}, {'TEXT': 'for'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'combining'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'techniques'}, {'TEXT': 'in'}, {'P

In [93]:
lp = tuple_search(l_one, l_two, m_one, m_two, l_five, "test.txt")

[[{'TEXT': '-'}, {'TEXT': 'infinite'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'k'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'information'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'data'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'geographic'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'geographic'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'graph'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'temporal'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'aware'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'intelligent'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}]]
['service discovery', 'network coding', 'resource management', 'mining techniques', 'video production', 'data types', 'traffic sign', 'similarity search', 'programming problems', 'information retrieval', 'encryption standard', 'data warehousing', 'security governance'

In [94]:
very_new_seed = set(['systems using', 'information theory', 'job scheduling', 'machine learning', 'face recognition', "machine learning", "query optimization", "RSA encryption", "tensor algebra", 'internet routers', 'storage devices', 'photo album', 'motion picture', 'dos defenses', 'resource retrieval', 'parallel algorithms', 'enterprise architectures', 'multicriteria decision', 'lossless image', 'recommendation systems', 'simulation models', 'television services', 'multimedia storage', 'intrusion detection', 'information retrieval', 'computational neuroscience', 'object retrieval', 'matrix quadtree', 'communication systems', 'computer vision', 'approximation algorithms', 'motion coordination', 'target tracking', 'hardware platforms', 'web server', 'message authentication', 'data mining', 'service differentiation', 'optimization techniques', 'testing tool', 'hand gesture', 'reinforcement\\break learning', 'information assurance', 'shape retrieval', 'routing algorithm', 'assessment system', 'face verification', 'video compression', 'security policy', 'model checking', 'botnet detection', 'sql solutions', 'copyright protection', 'software engineering', 'knowledge management', 'buffer overflows', 'security protocols', 'multimedia retrieval', 'image indexing', 'asset compiler', 'process analysis', 'belief revision', 'internet pornography', 'search engine', 'knowledge integration', 'feature extraction', 'access control', 'data structures', 'planning techniques', 'reinforcement learning', 'user interfaces', 'face identification', 'optimization problems', 'query processing', 'machine learning', 'resource allocation', 'face recognition', 'authorization languages', 'service discovery', 'network coding', 'resource management', 'mining techniques', 'video production', 'data types', 'traffic sign', 'similarity search', 'programming problems', 'information retrieval', 'encryption standard', 'data warehousing', 'security governance', 'security policies', 'base sublanguage', 'association patterns', 'resource discovery', 'markup language', 'software agents', 'data mining', 'data management', 'coverage standards', 'management systems', 'security management', 'connectionist networks', 'radar detectors', 'access controls', 'compression codes', 'information systems', 'security controls', 'market segmentation', 'query languages', 'security policy', 'retrieval techniques', 'security basics', 'structure visualization', 'power management', 'task scheduling', 'resources analysis', 'data models', 'index structures', 'systems security', 'olap queries', 'search engine', 'fusion algorithms', 'power system', 'simd algorithm', 'mining query', 'management techniques', 'access control', 'security risk', 'data reduction', 'video content', 'presentation methods', 'identity management', 'optimization problems', 'query processing', 'user interfaces', 'cache management', 'mining model'])

In [95]:
la, lb, lc, ld, ma, mb, le = patternSearch(very_new_seed, 'test.txt')

[[{'TEXT': '\n'}, {'TEXT': 'multimedia'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}], [{'TEXT': 'for'}, {'TEXT': 'multimedia'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'structures'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'atlas'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'spatial'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'combining'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'techniques'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'language'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'physical'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'peer'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'discovery'}, {'TEXT': 'and'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'biometrics'}, {'TEXT': 'for'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'learning'}, {'TEXT': 'and'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'ad

In [96]:
lp2 = tuple_search(la, lb, ma, mb, le, "test.txt")

[[{'TEXT': '\n'}, {'TEXT': 'multimedia'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}], [{'TEXT': 'for'}, {'TEXT': 'multimedia'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'structures'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'atlas'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'spatial'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'combining'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': 'techniques'}, {'TEXT': 'in'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'language'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '\n'}, {'TEXT': 'physical'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}], [{'TEXT': '-'}, {'TEXT': 'peer'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}]]
['entertainment applications', 'document analysis', 'intra-', 'cone tree', 'content adaptation', 'multicriteria decision', 'association analysis', 'document categorization', 'news systems', 'data types', 'content personalisation', 'lag model', 'information 